<a href="https://colab.research.google.com/github/y001003/system_quality_project/blob/main/notebooks/System_quality_Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회

https://dacon.io/competitions/official/235687/overview/description

## 1. Library Import & Data 불러오기

In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 77 kB/s 


In [8]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import catboost as cb
import lightgbm as lgb
from tqdm import tqdm

In [15]:
PATH = '/content/drive/MyDrive/project/data/system_quality_customer_complain_data/'
train_err  = pd.read_csv(PATH+'train_err_data_.csv')
id_error = train_err[['user_id','errtype']].values
# 불만을 제기한 사람 타겟값 생성
train_prob = pd.read_csv(PATH+'train_problem_data_.csv')

array([10000,    15])

## 2. 학습 데이터 생성

### 2_1 train_df 생성

In [17]:
#error 0으로 이루어진 데이터셋 생성
error = np.zeros((15000,42)) # 15000개 user_id, 41가지(maxnum 42) errtype 

for person_idx, err in tqdm(id_error):
    # (person_idx - 10000)에 해당하는 행에, 열 기준 errtype 값의 -1 에 해당하는 위치에 error값을 +1
    error[person_idx - 10000,err - 1] += 1
error 

100%|██████████| 16554663/16554663 [00:45<00:00, 361337.77it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [20]:
train_df = pd.DataFrame(error)
train_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,...,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


### 2_2 target 생성

In [27]:
# 0 으로 이루어진 열 생성
problem = np.zeros(15000)
# user_id 각각에 10000 빼서 저장
user_ids = train_prob['user_id'].unique() - 10000
# probelm[유저id번호] 에 1 저장
problem[user_ids] = 1

display(problem, user_ids, sum(problem))

array([0., 1., 0., ..., 1., 1., 0.])

array([ 9224, 13664,  5166, ...,  9114, 11505,  8822])

5000.0

In [29]:
target = pd.DataFrame(problem)
target

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


## 3. CatBoost 훈련


In [61]:
from sklearn.model_selection import train_test_split

In [62]:
# train validation 분리
X_train, X_val, y_train, y_val = train_test_split(train_df,target,test_size=0.2, random_state=42)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((12000, 42), (3000, 42), (12000, 1), (3000, 1))

In [63]:
y_train

,0
9839,0.0
9680,0.0
7093,0.0
11293,0.0
820,0.0
...,...
5191,1.0
13418,1.0
5390,1.0
860,1.0


In [64]:
model = cb.CatBoostClassifier(iterations=2,
                           learning_rate=0.01,
                           max_depth=10,
                           early_stopping_rounds=3
                           )
cb_model = model.fit(X_train,y_train)

0:	learn: 0.6886158	total: 72.5ms	remaining: 72.5ms
1:	learn: 0.6840796	total: 149ms	remaining: 0us


In [65]:
# Validation 데이터셋 예상
pred = cb_model.predict(X_val)

In [66]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred))  #confusion matrix 표시
print(precision_score(y_val, pred)) #precision(정밀도)
print(recall_score(y_val, pred))     #sensitivity(민감도), recall(재현율)
print(f1_score(y_val, pred))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred))

array([[1564,   69],
       [ 865,  502]])

0.8791593695271454
0.3672275054864667
0.5180598555211559
              precision    recall  f1-score   support

         0.0       0.64      0.96      0.77      1633
         1.0       0.88      0.37      0.52      1367

    accuracy                           0.69      3000
   macro avg       0.76      0.66      0.64      3000
weighted avg       0.75      0.69      0.66      3000

